# README

В пунктах 2-4 происходит удаление сущетсвующих таблиц, заполнение новых, тестирование, удаление тестовых данных.
Для единичной прогонки для новой таблицы необходимо выполнить пункты 5-8. В пункте 9 закрываем открытые соединения. Пункт 10 содержит замер скорости для получения подбора сумм.

# Начало

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect(':memory:')
c = conn.cursor()
v = conn.cursor()

In [3]:
pd.read_sql_query("SELECT 'OK'", conn)

,'OK'
0,OK


# СОЗДАЕМ ТАБЛИЦЫ

логика такая:

сначала инициализируем переменные с именами таблицы

потом удаляем существующие таблицы

потом создаем таблицы и нужную для работы программу логику:

1) создаем таблицу

2) создаем класс для представления в питоне, что по сути является частью промежуточной области.

3) определяем функцию, которая переводит данные из представления питона в хранилище данных

4) произвожу тестирование, перед основной работой программы внесенные данные почищу

In [4]:
CLIENTS = 'clients'
ACCOUNTS = 'accounts'
CARDS = 'cards'
TERMINALS = 'terminals'
TRANSACTION = 'transactions'
META = 'meta'

## drop tables.

In [5]:
# c.execute(f'drop table {CLIENTS}')

In [6]:
# c.execute(f'drop table {ACCOUNTS}')

In [7]:
# c.execute(f'drop table {CARDS}')

In [8]:
# c.execute(f'drop table {TERMINALS}')

In [9]:
# c.execute(f'drop table {TRANSACTION}')

In [10]:
# c.execute('commit')

In [11]:
# считываем данные для проверки, потом - удалим
data = pd.read_excel("passport.xlsx")
data.shape
stage = data

## TRANSACTION

In [12]:
#create TRANSACTION
c.execute(f"""
CREATE TABLE {TRANSACTION}
    (
     trans_id               varchar (20)        NULL,               -- 1      --int64 (8 знаков)
     trans_date             DATE                NULL,       -- 2      --datetime64[ns] yyyy-mm-dd hh:mm:ss
     card_num               varchar (20)        NULL,       -- 3      --object (20 знаков, числа )
     oper_type              varchar (40)        NULL,       -- 14     --object максимум 11 кирилицей
     amt                    number (9,2)        NULL,       -- 15     --float64 (2 занка после запятой)
     oper_result            varchar (20)        NULL,       -- 16     --object 7 знаков кирилицей
     terminal               varchar (10)        NULL,       -- 17     --object 8 знаков 3латиница и 5 цифр
     start_dt               DATE                NULL,
     end_dt                 DATE                NULL
    )
""")

In [13]:
class class_transactions:
    def insert(self, start_dt):
        return f"""
insert into {TRANSACTION}(
   trans_id
  ,trans_date
  ,card_num
  ,oper_type
  ,amt
  ,oper_result
  ,terminal
  ,start_dt
)
values (
   '{self.trans_id}'
  , strftime('%Y-%m-%d %H:%M:%S', '{self.trans_date}')
  ,'{self.card_num}'
  ,'{self.oper_type}'
  , {self.amt}
  ,'{self.oper_result}'
  ,'{self.terminal}'
  , strftime('%Y-%m-%d %H:%M:%S', '{start_dt}')
)"""
    
    def __init__(self, trans_id, trans_date, card_num, oper_type, amt, oper_result, terminal):
        self.trans_id = trans_id
        self.trans_date = trans_date
        self.card_num = card_num        
        self.oper_type = oper_type
        self.amt = amt
        self.oper_result = oper_result
        self.terminal = terminal
        
    def __str__(self):
        return 'trans_id=' + str(self.trans_id) + ", trans_date=" + str(self.trans_date) + ", card_num=" + str(self.card_num) + ", oper_type=" + str(self.oper_type) + ', amt=' + str(self.amt) + ', oper_result=' + str(self.oper_result) + ', terminal=' + str(self.terminal)


In [14]:
def fill_transaction(new_data, index):
    return class_transactions(new_data.iloc[index][0],new_data.iloc[index][1],new_data.iloc[index][2],
                        new_data.iloc[index][13],new_data.iloc[index][14],new_data.iloc[index][15],new_data.iloc[index][16])

In [15]:
data.iloc[0]['date']

Timestamp('2020-05-01 00:00:29')

In [16]:
q = fill_transaction(data, 0)

In [17]:
q.__str__()

'trans_id=1, trans_date=2020-05-01 00:00:29, card_num=59649132026167121328, oper_type=Пополнение, amt=31576.6, oper_result=Отказ, terminal=POS43792'

In [18]:
c.execute(q.insert(q.trans_date))

In [19]:
pd.read_sql_query(f"select * from {TRANSACTION}", conn)

,trans_id,trans_date,card_num,oper_type,amt,oper_result,terminal,start_dt,end_dt
0,1,2020-05-01 00:00:29,59649132026167121328,Пополнение,31576.6,Отказ,POS43792,2020-05-01 00:00:29,None


## TERMINALS

In [20]:
#create TERMINALS
c.execute(f"""
CREATE TABLE {TERMINALS}
    (
     terminal               varchar (10 )   NULL,       -- 17           --object 8 знаков 3латиница и 5 цифр
     terminal_type          varchar (3 )    NULL,       -- 18           --object (3 знака латиница) POS/ATM
     terminal_city          varchar (50 )   NULL,       -- 19           --object max 22 кирилицей
     terminal_address       varchar (100 )  NULL,       -- 20           --object max 45 кирилицей
     start_dt               date            NULL,
     end_dt                 date            NULL
    )
""")

In [21]:
class class_terminals:
    def insert(self, start_dt):
        return f"""
insert into {TERMINALS} (
  terminal
  ,terminal_type
  ,terminal_city
  ,terminal_address
  ,start_dt
)
values (
   '{self.terminal_id}'
  ,'{self.terminal_type}'
  ,'{self.terminal_city}'
  ,'{self.terminal_address}'
  ,strftime('%Y-%m-%d %H:%M:%S', '{start_dt}')
)"""
    def __eq__(self, other):
        return self.terminal_id == other.terminal_id and self.terminal_type == other.terminal_type and self.terminal_city == other.terminal_city and self.terminal_address == other.terminal_address
       
    def __init__(self, terminal_id, terminal_type, terminal_city, terminal_address):
        self.terminal_id = terminal_id
        self.terminal_type = terminal_type
        self.terminal_city = terminal_city
        self.terminal_address = terminal_address
    def __str__(self):
        return "terminal_id=" + str(self.terminal_id) + ', terminal_type=' + str(self.terminal_type) +', terminal_city='+ str(self.terminal_city) + ', terminal_address=' + str(self.terminal_address)

In [22]:
def fill_terminal(new_data, index):
    return class_terminals(new_data.iloc[index][16], new_data.iloc[index][17],new_data.iloc[index][18],new_data.iloc[index][19])

In [23]:
w = fill_terminal(data, 0)
print(w)

terminal_id=POS43792, terminal_type=POS, terminal_city=Славянск-на-Кубани, terminal_address=Славянск-на-Кубани, ул. Клецкая, д. 86


In [24]:
c.execute(w.insert(q.trans_date))

In [25]:
pd.read_sql_query(f"select * from {TERMINALS}", conn)

,terminal,terminal_type,terminal_city,terminal_address,start_dt,end_dt
0,POS43792,POS,Славянск-на-Кубани,"Славянск-на-Кубани, ул. Клецкая, д. 86",2020-05-01 00:00:29,None


## CARDS

In [26]:
#create CARDS
c.execute(f"""
CREATE TABLE {CARDS}
    (
     card_num               varchar (20 )       NOT NULL,   -- 3            --object (20 знаков, числа )
     account_num            varchar (20 )       NULL,       -- 4            --object (20 знаков, числа)
     start_dt               DATE                NOT NULL,
     end_dt                 DATE                NULL
    )
""")

In [27]:
class class_cards:
    def insert(self, start_dt):
        return f"""
insert into {CARDS} (
   card_num
  ,account_num
  ,start_dt
)
values (
   '{self.card_num}'
  ,'{self.account_num}'
  ,strftime('%Y-%m-%d %H:%M:%S', '{start_dt}')
)"""
    def __eq__(self, other):
        return self.card_num == other.card_num and self.account_num == other.account_num
    
    def __init__(self, card_num, account_num):
        self.card_num = card_num
        self.account_num = account_num
    def __str__(self):
        return 'card_num=' + str(self.card_num) + ', account_num=' + str(self.account_num)

In [28]:
def fill_card(new_data, index):
    return class_cards(new_data.iloc[index][2], new_data.iloc[index][3])

In [29]:
e = fill_card(data, 0)
print(e)

card_num=59649132026167121328, account_num=40817810000001139973


In [30]:
c.execute(e.insert(q.trans_date))

In [31]:
pd.read_sql_query(f"select * from {CARDS}", conn)

,card_num,account_num,start_dt,end_dt
0,59649132026167121328,40817810000001139973,2020-05-01 00:00:29,None


## ACCOUNTS

In [32]:
#create ACCOUNTS
c.execute(f"""
CREATE TABLE {ACCOUNTS}
    (
     account_num            varchar (20 )       NOT NULL,   -- 4            --object (20 знаков, числа)
     valid_to               date                NULL,       -- 5            --date yyyy-mm-dd
     client                 varchar (7 )        NULL,       -- 6            --object (7знаков, числа-)
     start_dt               DATE                NOT NULL,
     end_dt                 DATE                NULL
    )
""")

In [33]:
class class_accounts:
    def insert(self, start_dt):
        return f"""
insert into {ACCOUNTS} (
   account_num
  ,valid_to
  ,client 
  ,start_dt
)
values (
  '{self.account_num}'
  ,strftime('%Y-%m-%d %H:%M:%S', '{self.valid_to}')
  ,'{self.client}'
  ,strftime('%Y-%m-%d %H:%M:%S', '{start_dt}')
)"""
    def __eq__(self, other):
        return self.account_num == other.account_num and self.valid_to == other.valid_to and self.client == other.client
    
    def __init__(self, account_num, valid_to, client):
        self.account_num = account_num
        self.valid_to = valid_to
        self.client = client
        
    def __str__(self):
        return 'account_num=' + str(self.account_num) + ', valid_to=' + str(self.valid_to) + ', client=' + str(self.client)


In [34]:
def fill_account(new_data, index):
    return class_accounts(new_data.iloc[index][3],new_data.iloc[index][4],new_data.iloc[index][5])

In [35]:
r = fill_account(data, 0)
print(r)

account_num=40817810000001139973, valid_to=2036-01-16 00:00:00, client=3-95179


In [36]:
c.execute(r.insert(q.trans_date))

In [37]:
pd.read_sql_query(f"select * from {ACCOUNTS}", conn)

,account_num,valid_to,client,start_dt,end_dt
0,40817810000001139973,2036-01-16 00:00:00,3-95179,2020-05-01 00:00:29,None


## CLIENTS

In [38]:
#create CLIENTS
c.execute(f"""
CREATE TABLE {CLIENTS}
    (
     client_id              varchar (7 )        NOT NULL,   -- 6            --object (7знаков, числа-)
     last_name              varchar (40 )       NULL,       -- 7            --object кирилица
     first_name             varchar (40 )       NULL,       -- 8            --object кирилица
     patronymic             varchar (40 )       NULL,       -- 9            --object кирилица
     date_of_birth          date                NULL,       -- 10           --date yyyy-mm-dd
     passport_num           varchar (10 )       NULL,       -- 11           --int64 10 знаков
     passport_valid_to      date                NULL,       -- 12           --date yyyy-mm-dd
     phone                  varchar (11 )       NULL,       -- 13           --int64 11знаков
     start_dt               date                NOT NULL,
     end_dt                 date                NULL
    )
""")

In [39]:
class class_clients:
    def insert(self, start_dt):
        return f"""
insert into {CLIENTS} (
  client_id
  ,last_name
  , first_name
  , patronymic
  , date_of_birth
  , passport_num
  , passport_valid_to
  , phone
  , start_dt
)
values (
  '{self.client_id}'
  , '{self.last_name}'
  , '{self.first_name}'
  , '{self.patrinmic}'
  , strftime('%Y-%m-%d %H:%M:%S', '{self.date_of_birth}')
  , '{self.passport_num}'
  , strftime('%Y-%m-%d %H:%M:%S', '{self.passport_valid_to}')
  , '{self.phone}'
  , strftime('%Y-%m-%d %H:%M:%S', '{start_dt}')
)"""
    
    def __eq__(self, other):
        return self.client_id == other.client_id and    self.last_name == other.last_name and     self.first_name == other.first_name and    self.patrinmic == other.patrinmic and    self.date_of_birth == other.date_of_birth and    str(self.passport_num) == str(other.passport_num) and    self.passport_valid_to == other.passport_valid_to and    str(self.phone) == str(other.phone)
    
    def __init__(self, client_id, last_name, first_name, patrinmic, date_of_birth, passport_num, passport_valid_to, phone):
        self.client_id = client_id
        self.last_name = last_name
        self.first_name = first_name
        self.patrinmic = patrinmic
        self.date_of_birth = date_of_birth
        self.passport_num = passport_num
        self.passport_valid_to = passport_valid_to
        self.phone = phone
        
    def __str__(self):
        return 'client_id='+str(self.client_id) + ', last_name=' + str(self.last_name) + ', first_name=' + str(self.first_name)+    ', patrinmic=' + str(self.patrinmic) + ', date_of_birth=' + str(self.date_of_birth) + ', passport_num=' + str(self.passport_num)+    ', passport_valid_to=' + str(self.passport_valid_to) + ', phone=' + str(self.phone)

In [40]:
def fill_client(new_data, index):
    return class_clients(new_data.iloc[index][5],new_data.iloc[index][6],new_data.iloc[index][7],new_data.iloc[index][8],new_data.iloc[index][9],
                  new_data.iloc[index][10],new_data.iloc[index][11],new_data.iloc[index][12])

In [41]:
t = fill_client(data, 0)
print(t)

client_id=3-95179, last_name=Мисик, first_name=Сергей, patrinmic=Николаевич, date_of_birth=1938-06-25 00:00:00, passport_num=7076445954, passport_valid_to=2019-11-09 00:00:00, phone=79497481039


In [42]:
c.execute(t.insert(q.trans_date))

In [43]:
pd.read_sql_query(f"select * from {CLIENTS}", conn)

,client_id,last_name,first_name,patronymic,date_of_birth,passport_num,passport_valid_to,phone,start_dt,end_dt
0,3-95179,Мисик,Сергей,Николаевич,1938-06-25 00:00:00,7076445954,2019-11-09 00:00:00,79497481039,2020-05-01 00:00:29,None


## Создаем индексы

In [44]:
c.execute(f"create index IX_{TRANSACTION}_TRANS_ID on {TRANSACTION}(trans_id)")
c.execute(f"create index IX_{TERMINALS}_TERMINAL_END_DT on {TERMINALS}(terminal, end_dt)")
c.execute(f"create index IX_{CARDS}_card_num_end_dt on {CARDS}(card_num, end_dt)")
c.execute(f"create index IX_{ACCOUNTS}_account_num_end_dt on {ACCOUNTS}(account_num, end_dt)")
c.execute(f"create index IX_{CLIENTS}_client_id_end_dt on {CLIENTS}(client_id, end_dt)")
c.execute('commit')

## Создаем ограничения

не могу понять, как надо делать primaryKey на transactions, чтобы можно было сослаться на него в terminals и cards.
Есть вариант навесить его на (card_num, terminal_id, st_dt)
В terminals добавить card_num, а в cards -  terminal_id и нормально ссылать на ключ, но какое-то решение не очень на мой взгляд

## mata-info

с метой слегка другая логика - в ней я питоновское представление достаю из sql-таблицы.

In [45]:
c = conn.cursor()
v = conn.cursor()
c.execute(r"""select 'OK'""")
for row in c:
    print(row)

('OK',)


In [46]:
# c.execute(f'drop table {META}')

In [47]:
#create meta-info
c.execute(f"""
CREATE TABLE {META}
    (
     last_update_date   DATE   NULL,       
     passport_date      DATE   NULL,    
     account_date       DATE   NULL,       
     city_date          DATE   NULL, 
     sum_date           DATE   NULL,       
     start_dt           DATE   NULL,
     end_dt             DATE   NULL
    )
""")

вношу данные в таблицу

In [48]:
c.execute(f"""
insert into {META} (last_update_date, passport_date, account_date, city_date, sum_date, start_dt)
values (
    strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
    strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
    strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
    strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
    strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
    strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00')
)
""")

In [49]:
c.execute('commit')

In [50]:
class meta:
    def __init__(self, a,b,c,d,e):
        self.last_update_date = pd.Timestamp(a)
        self.passport_date = pd.Timestamp(b)
        self.account_date = pd.Timestamp(c)
        self.city_date = pd.Timestamp(d)
        self.sum_date = pd.Timestamp(e)

In [51]:
c.execute(f"select * from {META} where end_dt is null")

In [52]:
for row in c:
    print(row)
    meta_instance = meta(row[0],row[1],row[2],row[3],row[4])

('1990-01-01 00:00:00', '1990-01-01 00:00:00', '1990-01-01 00:00:00', '1990-01-01 00:00:00', '1990-01-01 00:00:00', '1990-01-01 00:00:00', None)


проверка

In [53]:
meta_instance.last_update_date

Timestamp('1990-01-01 00:00:00')

# чищу данные после тестирования

In [54]:
def test_case():
    """Подчищаем все данные из таблица, нужно для отладки программы"""
    c.execute(f'delete from {TERMINALS} where 1=1')
    c.execute(f'delete from {CARDS} where 1=1')
    c.execute(f'delete from {TRANSACTION} where 1=1')
    c.execute(f'delete from {ACCOUNTS} where 1=1')
    c.execute(f'delete from {CLIENTS} where 1=1')
    c.execute(f'delete from {META} where 1=1')
    c.execute(f"""
    insert into {META} (last_update_date, passport_date, account_date, city_date, sum_date, start_dt)
    values (
        strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
        strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
        strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
        strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
        strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00'),
        strftime('%Y-%m-%d %H:%M:%S', '1990-01-01 00:00:00')
    )
    """)
    c.execute('commit')

In [55]:
test_case()

# Чтение данных

In [56]:
data_path = 'transactions_01052020.xlsx'

In [57]:
import datetime
data = pd.read_excel(f"{data_path}")
data['date'] = data['date'].apply(lambda x: datetime.datetime.strptime(str(x).replace('.','-'), '%Y-%m-%d %H:%M:%S'))
data['account_valid_to'] = data['account_valid_to'].apply(lambda x: str(x).replace('.','-'))
data['date_of_birth'] = data['date_of_birth'].apply(lambda x: str(x).replace('.','-'))
data['passport_valid_to'] = data['passport_valid_to'].apply(lambda x: str(x).replace('.','-'))
stage = pd.DataFrame(columns = data.columns)

In [58]:
for i in range(len(data)):
    #считали построчно из источника данные в stage
    if(data.loc[i].date > meta_instance.last_update_date):
        stage.loc[i] = data.loc[i]

# Основная работа

In [59]:
c = conn.cursor()
v = conn.cursor()
c.execute(r"""select 'OK'""")
for row in c:
    print(row)

('OK',)


In [60]:
counter = 0
for i in range(len(stage)):
    # вероятнее всего надо сделать проверку, что stage.date > meta.date
    
    #вносим транзакцию
    transaction = fill_transaction(stage, i)
    print(transaction)
    #если дата транакции <= даты обновления мета-информации, то такую траназкцию пропускаем
    if (transaction.trans_date <= meta_instance.last_update_date):
        continue
    c.execute(transaction.insert(transaction.trans_date))
    
    
#   вносим терминал
    terminal = fill_terminal(stage, i)
    c.execute(f"select count(*) from {TERMINALS} where terminal='{terminal.terminal_id}'")
    for row in c:
        counter = row[0]
        
    if (counter == 0):
        c.execute(terminal.insert(transaction.trans_date))
    else :
        c.execute(f"select * from {TERMINALS} WHERE terminal='{terminal.terminal_id}' and end_dt is null")
        #тут будет одна сктрока - для которой отсутвует end_dt
        for row in c:
#             print(row)
            
            terminal_from_sql = class_terminals(row[0], row[1], row[2], row[3])
            if(terminal_from_sql != terminal):
                print("not equal")
                v.execute(f"""update {TERMINALS} set end_dt = strftime('%Y-%m-%d %H:%M:%S', '{transaction.trans_date}')
                          where terminal='{terminal.terminal_id}' and end_dt is null""")
                v.execute(terminal.insert(transaction.trans_date))
            
    #вносим карты
    card = fill_card(stage, i)
    c.execute(f"select count(*) from {CARDS} where card_num='{card.card_num}'")
    for row in c:
        counter = row[0]
    #сделал аналогично транзакциям
    if (counter == 0):
        c.execute(card.insert(transaction.trans_date))
    else :
        print(f"повторная карта {card.card_num}")
        c.execute(f"select * from {CARDS} WHERE card_num='{card.card_num}' AND end_dt is null")
        for row in c:
            card_from_sql = class_cards(row[0], row[1])
            if(card_from_sql != card):
                v.execute(f"""update {CARDS} set end_dt = strftime('%Y-%m-%d %H:%M:%S', '{transaction.trans_date}')
                          where card_num='{card.card_num}' and end_dt is null""")
                v.execute(card.insert(transaction.trans_date))
        
    #вносим аккаунты
    account = fill_account(stage, i)
    c.execute(f"select count(*) from {ACCOUNTS} where account_num='{account.account_num}'")
    for row in c:
        counter = row[0]
    if (counter == 0):
        c.execute(account.insert(transaction.trans_date))
    else :
        c.execute(f"select * from {ACCOUNTS} WHERE account_num='{account.account_num}' AND end_dt is null")
#         #тут будет одна сктрока - для которой отсутвует end_dt
        for row in c:
            account_from_sql = class_accounts(row[0], row[1], row[2])
            if(account_from_sql != account):
#                 print(account.account_num)
                v.execute(f"""update {ACCOUNTS} set end_dt = strftime('%Y-%m-%d %H:%M:%S', '{transaction.trans_date}')
                          where account_num='{account.account_num}' and end_dt is null""")
                v.execute(account.insert(transaction.trans_date))

    #вносим клиентов
    client = fill_client(stage, i)
    c.execute(f"select count(*) from {CLIENTS} where client_id='{client.client_id}'")
    for row in c:
        counter = row[0]
    if (counter == 0):
        c.execute(client.insert(transaction.trans_date))
    else:
        c.execute(f"select * from {CLIENTS} WHERE client_id='{client.client_id}' AND end_dt is null")
        for row in c:
            client_from_sql = class_clients(row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7])
            if(client_from_sql != client):
                v.execute(f"""update {CLIENTS} set end_dt = strftime('%Y-%m-%d %H:%M:%S', '{transaction.trans_date}')
                          where client_id='{client.client_id}' and end_dt is null""")
                v.execute(client.insert(transaction.trans_date))
                
    c.execute("commit")
    # v.execute('commit')

trans_id=459270924, trans_date=2020-05-01 00:00:29, card_num=59649132026167121328, oper_type=Пополнение, amt=31576.6, oper_result=Отказ, terminal=POS43792
trans_id=459270943, trans_date=2020-05-01 00:02:46, card_num=54236863445669263724, oper_type=Пополнение, amt=60041.1, oper_result=Отказ, terminal=POS75616
trans_id=459270968, trans_date=2020-05-01 00:05:14, card_num=45930894029725313464, oper_type=Снятие, amt=18474.46, oper_result=Успешно, terminal=POS28311
trans_id=459271057, trans_date=2020-05-01 00:07:05, card_num=47550893546403715714, oper_type=Пополнение, amt=915.26, oper_result=Успешно, terminal=ATM39555
trans_id=459271064, trans_date=2020-05-01 00:08:27, card_num=53036257457049008830, oper_type=Оплата, amt=75843.91, oper_result=Отказ, terminal=ATM16730
trans_id=459271161, trans_date=2020-05-01 00:09:16, card_num=42810999157069197448, oper_type=Оплата, amt=52415.15, oper_result=Успешно, terminal=POS64475
trans_id=459271162, trans_date=2020-05-01 00:09:59, card_num=4508006371660

In [61]:
meta_instance.last_update_date = stage['date'].max()

# Построение отчета

## определяем вспомагательные функции

Логика следующая:
1) определяем две функции для корректной состыковки таблиц на основе start_dt и end_dt
2) определяем функцию, которая красиво будет переписывать строку из sql-запросов в .txt файл

In [62]:
def getter_cards_accounts_clients():
    '''
возвращает условие для корректного джоина
    '''
    return f"""
CAST(strftime('%s', {TRANSACTION}.trans_date) as integer)  >= CAST(strftime('%s', {CARDS}.start_dt ) as integer)
   and CAST(strftime('%s', {TRANSACTION}.trans_date) as integer) < CAST(strftime('%s', coalesce({CARDS}.end_dt, strftime('%Y-%m-%d %H:%M:%S', '9999-12-12 00:00:00'))) as integer)
and
CAST(strftime('%s', {TRANSACTION}.trans_date) as integer)  >= CAST(strftime('%s', {ACCOUNTS}.start_dt ) as integer)
   and CAST(strftime('%s', {TRANSACTION}.trans_date) as integer) < CAST(strftime('%s', coalesce({ACCOUNTS}.end_dt, strftime('%Y-%m-%d %H:%M:%S', '9999-12-12 00:00:00'))) as integer)
   and
CAST(strftime('%s', {TRANSACTION}.trans_date) as integer)  >= CAST(strftime('%s', {CLIENTS}.start_dt ) as integer)
   and CAST(strftime('%s', {TRANSACTION}.trans_date) as integer) < CAST(strftime('%s', coalesce({CLIENTS}.end_dt, strftime('%Y-%m-%d %H:%M:%S', '9999-12-12 00:00:00'))) as integer)
    """

In [63]:
def getter_cards_accounts_clients_terminals():
    '''
возвращает условие для корректного джоина
    '''
    return f"""
CAST(strftime('%s', {TRANSACTION}.trans_date) as integer)  >= CAST(strftime('%s', {CARDS}.start_dt ) as integer)
   and CAST(strftime('%s', {TRANSACTION}.trans_date) as integer) < CAST(strftime('%s', coalesce({CARDS}.end_dt, strftime('%Y-%m-%d %H:%M:%S', '9999-12-12 00:00:00'))) as integer)
and
CAST(strftime('%s', {TRANSACTION}.trans_date) as integer)  >= CAST(strftime('%s', {ACCOUNTS}.start_dt ) as integer)
   and CAST(strftime('%s', {TRANSACTION}.trans_date) as integer) < CAST(strftime('%s', coalesce({ACCOUNTS}.end_dt, strftime('%Y-%m-%d %H:%M:%S', '9999-12-12 00:00:00'))) as integer)
and
CAST(strftime('%s', {TRANSACTION}.trans_date) as integer)  >= CAST(strftime('%s', {CLIENTS}.start_dt ) as integer)
   and CAST(strftime('%s', {TRANSACTION}.trans_date) as integer) < CAST(strftime('%s', coalesce({CLIENTS}.end_dt, strftime('%Y-%m-%d %H:%M:%S', '9999-12-12 00:00:00'))) as integer)
and
CAST(strftime('%s', {TRANSACTION}.trans_date) as integer)  >= CAST(strftime('%s', {TERMINALS}.start_dt ) as integer)
   and CAST(strftime('%s', {TRANSACTION}.trans_date) as integer) < CAST(strftime('%s', coalesce({TERMINALS}.end_dt, strftime('%Y-%m-%d %H:%M:%S', '9999-12-12 00:00:00'))) as integer)
    """

In [64]:
def row_to_fraud_str(row):
    """преобразуем полученную строку в строку для отчета"""
    return (f"дата транзакции '{row[0]}'" +
            f", номер паспорта клиента '{row[1]}'" +
            f", ФИО клиента '{row[2]} {row[3]} {row[4]}'" +
            f", номер телефона '{row[5]}'" +
            f", описание типа предпалагаемого мошеничества '{row[6]}'" +
            f", время построения отчета '{row[7]}'" +
            "\n")

In [65]:
import os.path
if os.path.exists('report.csv'):
    reportDf = pd.read_csv('report.csv')
else:
    reportDf = pd.DataFrame(columns=['fraud_dt', 'passport', 'last_name', 'first_name', 'patronymic', 'phone', 'fraud_type', 'report_dt'])

## Выполняем поиск нарушений

Логика следующая:
1) выполняем select-запрос, который находит все нарушения по типу

2) записываем полученные результаты в .txt файл и выводим в output (для наглядности)

3) обновляем мета-информацию:

Для нарушений по паспорту и договору meta информация содержит максимальную дату из stage.

Для разных городов и подбора сумм логика обновления соответсвующей meta-информации немножного другая. Тк нам необходимо учитывать для функции LAG данные с прошлого дня. Поэтому в meta информацию мы кладем 23:40 или 23:00. Старые данные не будут учитываться дважды, тк в конечном select мы делаем условие вывода данные только с новой даты с помощью функции get_date(meta_insatnce.last_update_date) - она вернет полночь наступившей даты, тк мета информация уже была обновлена.

### просроченный паспорт

In [66]:
passportDf = pd.read_sql_query(f"""
select
    fraud_dt
    , passport
    , last_name
    , first_name
    , patronymic
    , phone
    , fraud_type
    , report_dt
from (
    select
    trans_date as fraud_dt,
    passport_num as passport,
    last_name
    ,first_name,
    patronymic,
    phone,
    'passport is outdated' as fraud_type,
    strftime('%Y-%m-%d %H:%M:%S', 'now') as report_dt
    , case
        when CAST(strftime('%s', passport_valid_to, '+1 day') as integer) - CAST(strftime('%s', trans_date) as integer) > 0 then 0 --добавляем 1 день к дате паспорта
        else 1
    end as flag
    from {TRANSACTION}
        left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
        left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
        left join {CLIENTS} on  {ACCOUNTS}.client = {CLIENTS}.client_id
    where CAST(strftime('%s', trans_date) as integer)  > CAST(strftime('%s', strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.passport_date}')) as integer)
    and {getter_cards_accounts_clients()}
)
where flag = 1
""", conn)
passportDf

,fraud_dt,passport,last_name,first_name,patronymic,phone,fraud_type,report_dt


In [67]:
reportDf = pd.concat([reportDf, passportDf], ignore_index=True)
reportDf.to_csv('report.csv')

In [68]:
#обновляем информацию меты. В качестве максимальной даты берем время самого последнего запроса из stage.
meta_instance.passport_date = meta_instance.last_update_date
meta_instance.passport_date

Timestamp('2020-05-01 23:58:55')

### просроченный договор

In [69]:
accountDf = pd.read_sql_query(f"""
select
    fraud_dt
    , passport
    , last_name
    , first_name
    , patronymic
    , phone
    , fraud_type
    , report_dt
from (
    select
        trans_date as fraud_dt,
        passport_num as passport,
        last_name,
        first_name,
        patronymic,
        phone,
        'account is outdated' as fraud_type,
        strftime('%Y-%m-%d %H:%M:%S', 'now') as report_dt,
        trans_date
        , passport_valid_to
        , valid_to - trans_date
        , case
            when CAST(strftime('%s', valid_to, '+1 day') as integer) - CAST(strftime('%s', trans_date) as integer) > 0 then 0 --добавляем 1 день к дате паспорта
            else 1
        end as flag
    from {TRANSACTION}
    left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
    left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
    left join {CLIENTS} on  {ACCOUNTS}.client = {CLIENTS}.client_id
    where CAST(strftime('%s', trans_date) as integer)  > CAST(strftime('%s', strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.account_date}')) as integer)
    and {getter_cards_accounts_clients()}
)
where flag = 1
""", conn)
accountDf

,fraud_dt,passport,last_name,first_name,patronymic,phone,fraud_type,report_dt
0,2020-05-01 02:45:05,5263317668,Петропуло,Евгений,Павлович,79714131751,account is outdated,2022-09-04 09:39:04
1,2020-05-01 04:54:37,8108438217,Антощук,Игорь,Владимирович,79451696230,account is outdated,2022-09-04 09:39:04
2,2020-05-01 05:10:50,9600352239,Гуляков,Михаил,Александрович,79529669124,account is outdated,2022-09-04 09:39:04
3,2020-05-01 12:37:08,5670092603,Поджарый,Роман,Константинович,79410212254,account is outdated,2022-09-04 09:39:04
4,2020-05-01 14:18:08,3904347595,Фель,Евгений,Никитович,79692321575,account is outdated,2022-09-04 09:39:04
5,2020-05-01 14:40:41,6403074379,Ачентов,Антон,Дмитриевич,79458324907,account is outdated,2022-09-04 09:39:04


In [70]:
reportDf = pd.concat([reportDf, accountDf], ignore_index=True)
reportDf.to_csv('report.csv')

In [71]:
#обновляем информацию меты
meta_instance.account_date = meta_instance.last_update_date
meta_instance.account_date

Timestamp('2020-05-01 23:58:55')

### Разные города

In [72]:
meta_instance.city_date

Timestamp('1990-01-01 00:00:00')

In [73]:
def get_date(q):
    """получаем полночь переданной даты"""
    if len(str(q.month)) < 2:
        MONTH = '0' + str(q.month)
    else:
        MONTH = str(q.month)

    if len(str(q.day)) < 2:
        DAY = '0' + str(q.day)
    else:
        DAY = str(q.day)
    return pd.Timestamp(str(q.year) + '-' + MONTH + '-' + DAY + ' 00:00:00')

In [74]:
cityDf = pd.read_sql_query(f"""
select
    fraud_dt,
    passport,
    last_name,
    first_name,
    patronymic,
    phone,
    'city fraud' as fraud_type,
    strftime('%Y-%m-%d %H:%M:%S', 'now') as report_dt
from
(
    select
        t.*
    from
    (
        select
            trans_id,
            trans_date
            , {ACCOUNTS}.client as client
            , trans_date as dt
             , LAG (trans_date , 1 , strftime('%Y-%m-%d %H:%M:%S', '1900.01.01 10:10:10')) OVER (PARTITION by client order by trans_date)  as lag_dt
             , CAST(strftime('%s', trans_date) as integer) - CAST(strftime('%s', LAG (trans_date , 1 , strftime('%Y-%m-%d %H:%M:%S', '1900.01.01 10:10:10')) OVER (PARTITION by client order by trans_date)) as integer) as diff --разница в секундах
            , terminal_city
            , LAG (terminal_city, 1, null) OVER (PARTITION by client order by trans_date) as city2
            , trans_date as fraud_dt
            , passport_num as passport
            , last_name
            , first_name
            , patronymic
            , phone
        from {TRANSACTION}
            left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
            left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
            left join {CLIENTS} on  {ACCOUNTS}.client = {CLIENTS}.client_id
            left join {TERMINALS} on {TERMINALS}.terminal = {TRANSACTION}.terminal
        where trans_date > strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.city_date}')
        and {getter_cards_accounts_clients_terminals()}
        order by client_id, trans_date
    ) t
    where diff <= 60 * 60
        and terminal_city != city2
        and trans_date > strftime('%Y-%m-%d %H:%M:%S', '{get_date(meta_instance.last_update_date)}')
) ret
""", conn)
cityDf

,fraud_dt,passport,last_name,first_name,patronymic,phone,fraud_type,report_dt
0,2020-05-01 06:43:17,8611132511,Фель,Константин,Васильевич,79727028539,city fraud,2022-09-04 09:39:04
1,2020-05-01 14:20:50,8611132511,Фель,Константин,Васильевич,79727028539,city fraud,2022-09-04 09:39:04
2,2020-05-01 19:39:12,5142842911,Барецкий,Олег,Олегович,79810391232,city fraud,2022-09-04 09:39:04
3,2020-05-01 22:14:53,5142842911,Барецкий,Олег,Олегович,79810391232,city fraud,2022-09-04 09:39:04
4,2020-05-01 23:11:29,5142842911,Барецкий,Олег,Олегович,79810391232,city fraud,2022-09-04 09:39:04
...,...,...,...,...,...,...,...,...
221,2020-05-01 09:10:27,5263317668,Петропуло,Евгений,Павлович,79714131751,city fraud,2022-09-04 09:39:04
222,2020-05-01 09:31:37,5263317668,Петропуло,Евгений,Павлович,79714131751,city fraud,2022-09-04 09:39:04
223,2020-05-01 11:59:52,5263317668,Петропуло,Евгений,Павлович,79714131751,city fraud,2022-09-04 09:39:04
224,2020-05-01 12:41:44,5263317668,Петропуло,Евгений,Павлович,79714131751,city fraud,2022-09-04 09:39:04


In [75]:
reportDf = pd.concat([reportDf, cityDf], ignore_index=True)
reportDf.to_csv('report.csv')

In [76]:
def date_minus_hour(q):
    """Получаем дататайм для даты и времени 23:00:00"""
    return pd.Timestamp(str(q.year) + '-' + str(q.month) + '-' + str(q.day) + ' 23:00:00')

In [77]:
#обновляем мета-информацию. В качестве максимальной даты берем время самого последнего запроса из stage.
max_trans_date = pd.read_sql_query(f"""
select max(trans_date) from {TRANSACTION}
""", conn)
max_trans_date['max(trans_date)'] = max_trans_date['max(trans_date)'].apply(lambda x: datetime.datetime.strptime(str(x).replace('.','-'), '%Y-%m-%d %H:%M:%S'))
ts = max_trans_date['max(trans_date)'][0]
meta_instance.city_date = date_minus_hour(ts)

### Попытка подбора сумм

In [78]:
sumDf = pd.read_sql_query(f"""
select
    trans_date as fraud_dt,
    passport,
    last_name,
    first_name,
    patronymic,
    phone,
    'summ fraud' as fraud_type,
    strftime('%Y-%m-%d %H:%M:%S', 'now') as report_dt
from (
    select
        trans_date,
        oper_result,
        trans_id,
        clt,
        amt1,
        amt2,
        amt3,
        amt4,
        CAST(strftime('%s', dt1) as integer) - CAST(strftime('%s', dt4) as integer) as diff_sec,
        passport,
        last_name,
        first_name,
        patronymic,
        phone
    from(
        select
            trans_date,
            trans_id,
            client_id as clt,
            oper_result,
            amt as amt1,
            LAG(AMT, 1, null) OVER (PARTITION by client order by trans_date) as amt2,
            LAG(AMT, 2, null) OVER (PARTITION by client order by trans_date) as amt3,
            LAG(AMT, 3, null) OVER (PARTITION by client order by trans_date) as amt4,
            trans_date as dt1,
            LAG(trans_date, 1, null) OVER (PARTITION by client order by trans_date) as dt2,
            LAG(trans_date, 2, null) OVER (PARTITION by client order by trans_date) as dt3,
            LAG(trans_date, 3, null) OVER (PARTITION by client order by trans_date) as dt4,
            client_id,
            passport_num as passport,
            last_name,
            first_name,
            patronymic,
            phone
        from
        (
            select
                *
            from {TRANSACTION}
                left join {CARDS} on {TRANSACTION}.card_num = {CARDS}.card_num
                left join {ACCOUNTS} on {CARDS}.account_num = {ACCOUNTS}.account_num
                left join {CLIENTS} on {ACCOUNTS}.client = {CLIENTS}.client_id
                left join {TERMINALS} on {TERMINALS}.terminal = {TRANSACTION}.terminal
                where trans_date > strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.sum_date}')
                and {getter_cards_accounts_clients_terminals()}
            order by client_id, trans_date
        ) t
    ) tmp
) res
where diff_sec <= 20 * 60
    and amt1 < amt2
    and amt2 < amt3
    and amt3 < amt4
    and oper_result='Успешно'
    and CAST(strftime('%s', trans_date) as integer) > CAST(strftime('%s', '{get_date(meta_instance.last_update_date)}') as integer)
""", conn)
sumDf

,fraud_dt,passport,last_name,first_name,patronymic,phone,fraud_type,report_dt


In [79]:
reportDf = pd.concat([reportDf, sumDf], ignore_index=True)
reportDf.to_csv('report.csv')

In [80]:
def date_minus_20_minutes(q):
    """Получаем датадайм для даты и времени 23:40:00"""
    if len(str(q.month)) < 2:
        MONTH = '0' + str(q.month)
    else:
        MONTH = str(q.month)

    if len(str(q.day)) < 2:
        DAY = '0' + str(q.day)
    else:
        DAY = str(q.day)
    return pd.Timestamp(str(q.year) + '-' + MONTH + '-' + DAY + ' 23:40:00')

In [81]:
#обновляем мета-информацию. В качестве максимальной даты берем время самого последнего запроса из stage.
max_trans_date = pd.read_sql_query(f"""
select max(trans_date) from {TRANSACTION}
""", conn)
max_trans_date['max(trans_date)'] = max_trans_date['max(trans_date)'].apply(lambda x: datetime.datetime.strptime(str(x).replace('.','-'), '%Y-%m-%d %H:%M:%S'))
ts = max_trans_date['max(trans_date)'][0]

meta_instance.sum_date = date_minus_20_minutes(ts)

# Вносим мету в бд

In [82]:
c.execute(f"""
update {META}
set end_dt = strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.last_update_date}')
where  end_dt is null""")

In [83]:
c.execute(f"""
insert into {META} (last_update_date, passport_date, account_date, city_date, sum_date, start_dt)
values (
    strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.last_update_date}'),
    strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.passport_date}'),
    strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.account_date}'),
    strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.city_date}'),
    strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.sum_date}'),
    strftime('%Y-%m-%d %H:%M:%S', '{meta_instance.last_update_date}')
)
""")

In [84]:
c.execute('commit')

# Конец
Закрываем все соединения

In [85]:
c.close()
v.close()